<a href="https://colab.research.google.com/github/alen-ru/PythonUrban2023/blob/main/PythonUrban_final_task_%D0%A0%D1%83%D1%81%D0%B0%D0%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Итоговый проект. Улицы. Объекты культурного наследия.

In [22]:
# TODO собрать установку всех необходимых модулей в одном месте
!pip install geopandas mapclassify
!pip install osmnx  # устанавливаем модуль osmnx

In [23]:
# TODO собрать импорты всех модулей в одном месте
import geopandas as gpd
import osmnx as ox

In [55]:
TILES = "CartoDB positron"  # Название подложки для карт

# TODO указать любой район Санкт-Петербург из OSM https://wiki.openstreetmap.org/wiki/RU:%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3/%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D1%8B
TERRITORY_NAME = 'Адмиралтейский район, Санкт-Петербург'  # название территории для которой будут строиться слои

# TODO указать ссылку на файл из вашего github репозитория, которая начинается с https://raw.githubusercontent.com/
KGIOP_FILE_URL = "https://raw.githubusercontent.com/alen-ru/PythonUrban2023/main/geojson_layers/kgiop_objects.geojson"  # ссылка на слой с объектами культурного наследия
STREETS_FILE_URL = "https://raw.githubusercontent.com/alen-ru/PythonUrban2023/main/geojson_layers/streets.geojson"  # ссылка на слой с улицами

# Территория

### Загрузка территории из OSM (Extract)

In [25]:
# TODO загрузить geodataframe с геометрией для территории TERRITORY_NAME
district = ox.geocode_to_gdf(TERRITORY_NAME)
district

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name
0,"POLYGON ((30.25023 59.90129, 30.25112 59.90103...",59.941057,59.896031,30.342678,30.250229,181631755,relation,1114193,59.918443,30.302795,boundary,administrative,10,0.41336,administrative,Admiralteysky District,"Admiralteysky District, Сенной округ, Saint Pe..."


In [20]:
district.explore()

## Улицы

### Загрузка файла с улицами (Extract)

In [49]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
file_url = STREETS_FILE_URL

district = ox.geocode_to_gdf(TERRITORY_NAME)
gdf_streets = gpd.read_file(file_url, mask=district)

gdf_streets


,name,geometry
0,набережная Обводного канала,"LINESTRING (3368853.534 8379613.768, 3368894.2..."
1,переулок Гривцова,"LINESTRING (3374917.530 8383417.375, 3374910.2..."
2,Люблинский переулок,"LINESTRING (3372221.049 8381344.575, 3372211.4..."
3,Бумажная улица,"LINESTRING (3369358.324 8379129.066, 3369358.6..."
4,13-я Красноармейская улица,"LINESTRING (3372683.994 8380856.184, 3372703.9..."
...,...,...
1184,Малая Митрофаньевская улица,"LINESTRING (3374305.629 8376701.087, 3374290.3..."
1185,Малая Митрофаньевская улица,"LINESTRING (3374263.773 8376690.877, 3374279.6..."
1186,Парфёновская улица,"LINESTRING (3374321.804 8378141.245, 3374321.6..."
1187,набережная реки Фонтанки,"LINESTRING (3375466.001 8382603.187, 3375113.7..."


In [47]:
gdf_streets.explore(tiles="CartoDB positron")

### Обработка данных с улицами (Transform)

In [50]:
# TODO сгруппировать и объединить геометрии с одинаковыми названиями
gdf_streets.dissolve(by='name')

gdf_streets


,name,geometry
0,набережная Обводного канала,"LINESTRING (3368853.534 8379613.768, 3368894.2..."
1,переулок Гривцова,"LINESTRING (3374917.530 8383417.375, 3374910.2..."
2,Люблинский переулок,"LINESTRING (3372221.049 8381344.575, 3372211.4..."
3,Бумажная улица,"LINESTRING (3369358.324 8379129.066, 3369358.6..."
4,13-я Красноармейская улица,"LINESTRING (3372683.994 8380856.184, 3372703.9..."
...,...,...
1184,Малая Митрофаньевская улица,"LINESTRING (3374305.629 8376701.087, 3374290.3..."
1185,Малая Митрофаньевская улица,"LINESTRING (3374263.773 8376690.877, 3374279.6..."
1186,Парфёновская улица,"LINESTRING (3374321.804 8378141.245, 3374321.6..."
1187,набережная реки Фонтанки,"LINESTRING (3375466.001 8382603.187, 3375113.7..."


### Сохранение слоя с улицами (Load)

In [60]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry
rename_column = {
    "name": "Название"
}
gdf_streets.rename(columns=rename_column, inplace=True)

# TODO для того чтобы переименовать индекс, нужно обратиться и нему и вызвать от него метод rename (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.rename.html)
gdf_streets.index.rename('Индекс', inplace= True )

gdf_streets

,Название,geometry
Индекс,,
0,набережная Обводного канала,"LINESTRING (3368853.534 8379613.768, 3368894.2..."
1,переулок Гривцова,"LINESTRING (3374917.530 8383417.375, 3374910.2..."
2,Люблинский переулок,"LINESTRING (3372221.049 8381344.575, 3372211.4..."
3,Бумажная улица,"LINESTRING (3369358.324 8379129.066, 3369358.6..."
4,13-я Красноармейская улица,"LINESTRING (3372683.994 8380856.184, 3372703.9..."
...,...,...
1184,Малая Митрофаньевская улица,"LINESTRING (3374305.629 8376701.087, 3374290.3..."
1185,Малая Митрофаньевская улица,"LINESTRING (3374263.773 8376690.877, 3374279.6..."
1186,Парфёновская улица,"LINESTRING (3374321.804 8378141.245, 3374321.6..."


In [52]:
# TODO сохранить слой в географической проекции в формате GeoJSON
gdf_streets.to_crs(4326).to_file('streets.geojson', driver='GeoJSON')


## Объекты культурного наследия

### Загрузка объектов культурного наследия (Extract)

In [56]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
file_url = KGIOP_FILE_URL

district = ox.geocode_to_gdf(TERRITORY_NAME)
gdf_kgiop = gpd.read_file(file_url, mask=district)

gdf_kgiop



,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry
0,1,—,Здание Консисторского управления Могилевской Р...,1870-1873; 1878-1879; 1896-1897; 1900-1902,"1-я Красноармейская ул., 11, лит. А, Б",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-22 от 21.07.2009,Памятник,POINT (3374338.001 8381192.049)
1,2,—,Здание манежа (экзерциргауса) лейб-гвардии Изм...,1795-1797,"1-я Красноармейская ул., 13; Измайловский пр.,...",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 141-47 от 02.07.1997,Памятник,POINT (3373995.026 8381101.886)
2,3,—,"Дом, где в начале 1895 г. Ленин В.И. встречалс...",—,"1-я Красноармейская ул., 22",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 141-47 от 02.07.1997,Памятник,POINT (3374182.710 8381041.925)
3,4,—,Трансформаторная подстанция,1906-1907,"11-я Красноармейская ул., 28",объект культурного наследия регионального знач...,Решение исполкома Ленгорсовета № 963 от 05.12....,Памятник,POINT (3372847.990 8380161.229)
4,5,—,Манеж графа Г.И. Рибопьера (Здание Санкт-Петер...,1887; 1891,"2-я Красноармейская ул., 12",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-26 от 15.09.2009,Памятник,POINT (3374362.046 8380882.255)
...,...,...,...,...,...,...,...,...,...
1017,9601,—,Здание (многоквартирный дом),1857-1858,"Фонтанки р. наб., 128, лит. А",выявленный объект культурного наследия,Распоряжение КГИОП от 17.06.2019 № 289-р,Памятник,POINT (3373574.015 8381544.720)
1018,9602,—,Здание (многоквартирный дом),1866,"Фонтанки р. наб., 132, лит. З",выявленный объект культурного наследия,Распоряжение КГИОП от 17.06.2019 № 290-р,Памятник,POINT (3373358.946 8381445.446)
1019,9608,—,"Участок сохранившегося культурного слоя, вмеща...",—,"квартал, образованный пер. Бойцова, наб. р. Фо...",выявленный объект культурного наследия,Распоряжение КГИОП от 06.08.2019 № 450-р,Памятник,POINT (3374012.280 8382428.690)
1020,9624,—,Депо комплекса построек Товарной станции Варша...,—,"Обводного кан. наб., 118, корп. 12, лит. Ч",выявленный объект культурного наследия,Распоряжение КГИОП от 22.08.2019 № 507-р,Памятник,POINT (3373240.947 8378866.136)


In [57]:
gdf_kgiop.explore(tiles="CartoDB positron")

### Обработка объектов культурного наследия (Transform)

In [58]:
# TODO добавить два столбца lon и lat, в которых будут долгота и широта
gdf_kgiop["lon"] = gdf_kgiop.to_crs(3857).geometry.centroid.to_crs(4326).x
gdf_kgiop["lat"] = gdf_kgiop.to_crs(3857).geometry.centroid.to_crs(4326).y
gdf_kgiop

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry,lon,lat
0,1,—,Здание Консисторского управления Могилевской Р...,1870-1873; 1878-1879; 1896-1897; 1900-1902,"1-я Красноармейская ул., 11, лит. А, Б",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-22 от 21.07.2009,Памятник,POINT (3374338.001 8381192.049),30.312194,59.916595
1,2,—,Здание манежа (экзерциргауса) лейб-гвардии Изм...,1795-1797,"1-я Красноармейская ул., 13; Измайловский пр.,...",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 141-47 от 02.07.1997,Памятник,POINT (3373995.026 8381101.886),30.309113,59.916189
2,3,—,"Дом, где в начале 1895 г. Ленин В.И. встречалс...",—,"1-я Красноармейская ул., 22",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 141-47 от 02.07.1997,Памятник,POINT (3374182.710 8381041.925),30.310799,59.915919
3,4,—,Трансформаторная подстанция,1906-1907,"11-я Красноармейская ул., 28",объект культурного наследия регионального знач...,Решение исполкома Ленгорсовета № 963 от 05.12....,Памятник,POINT (3372847.990 8380161.229),30.298809,59.911953
4,5,—,Манеж графа Г.И. Рибопьера (Здание Санкт-Петер...,1887; 1891,"2-я Красноармейская ул., 12",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-26 от 15.09.2009,Памятник,POINT (3374362.046 8380882.255),30.312410,59.915200
...,...,...,...,...,...,...,...,...,...,...,...
1017,9601,—,Здание (многоквартирный дом),1857-1858,"Фонтанки р. наб., 128, лит. А",выявленный объект культурного наследия,Распоряжение КГИОП от 17.06.2019 № 289-р,Памятник,POINT (3373574.015 8381544.720),30.305331,59.918183
1018,9602,—,Здание (многоквартирный дом),1866,"Фонтанки р. наб., 132, лит. З",выявленный объект культурного наследия,Распоряжение КГИОП от 17.06.2019 № 290-р,Памятник,POINT (3373358.946 8381445.446),30.303399,59.917736
1019,9608,—,"Участок сохранившегося культурного слоя, вмеща...",—,"квартал, образованный пер. Бойцова, наб. р. Фо...",выявленный объект культурного наследия,Распоряжение КГИОП от 06.08.2019 № 450-р,Памятник,POINT (3374012.280 8382428.690),30.309268,59.922163
1020,9624,—,Депо комплекса построек Товарной станции Варша...,—,"Обводного кан. наб., 118, корп. 12, лит. Ч",выявленный объект культурного наследия,Распоряжение КГИОП от 22.08.2019 № 507-р,Памятник,POINT (3373240.947 8378866.136),30.302339,59.906120


### Сохранение слоя с объектами культурного наследия (Load)

In [62]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry
rename_columns = {
    "id": "Индекс",
    "ensemble_name": "Название ансамбля",
    "object_name": "Название объекта",
    "occurrence_time": "Время появления",
    "object_location": "Местонахождение",
    "historical_category": "Историческая категория",
    "normative_act": "Нормативный акт",
    "object_type": "Тип объекта",
    "lon": "Долгота",
    "lat": "Широта"
}
gdf_kgiop.rename(columns=rename_columns, inplace=True)

gdf_kgiop

,Индекс,Название ансамбля,Название объекта,Время появления,Местонахождение,Историческая категория,Нормативный акт,Тип объекта,geometry,Долгота,Широта
0,1,—,Здание Консисторского управления Могилевской Р...,1870-1873; 1878-1879; 1896-1897; 1900-1902,"1-я Красноармейская ул., 11, лит. А, Б",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-22 от 21.07.2009,Памятник,POINT (3374338.001 8381192.049),30.312194,59.916595
1,2,—,Здание манежа (экзерциргауса) лейб-гвардии Изм...,1795-1797,"1-я Красноармейская ул., 13; Измайловский пр.,...",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 141-47 от 02.07.1997,Памятник,POINT (3373995.026 8381101.886),30.309113,59.916189
2,3,—,"Дом, где в начале 1895 г. Ленин В.И. встречалс...",—,"1-я Красноармейская ул., 22",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 141-47 от 02.07.1997,Памятник,POINT (3374182.710 8381041.925),30.310799,59.915919
3,4,—,Трансформаторная подстанция,1906-1907,"11-я Красноармейская ул., 28",объект культурного наследия регионального знач...,Решение исполкома Ленгорсовета № 963 от 05.12....,Памятник,POINT (3372847.990 8380161.229),30.298809,59.911953
4,5,—,Манеж графа Г.И. Рибопьера (Здание Санкт-Петер...,1887; 1891,"2-я Красноармейская ул., 12",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-26 от 15.09.2009,Памятник,POINT (3374362.046 8380882.255),30.312410,59.915200
...,...,...,...,...,...,...,...,...,...,...,...
1017,9601,—,Здание (многоквартирный дом),1857-1858,"Фонтанки р. наб., 128, лит. А",выявленный объект культурного наследия,Распоряжение КГИОП от 17.06.2019 № 289-р,Памятник,POINT (3373574.015 8381544.720),30.305331,59.918183
1018,9602,—,Здание (многоквартирный дом),1866,"Фонтанки р. наб., 132, лит. З",выявленный объект культурного наследия,Распоряжение КГИОП от 17.06.2019 № 290-р,Памятник,POINT (3373358.946 8381445.446),30.303399,59.917736
1019,9608,—,"Участок сохранившегося культурного слоя, вмеща...",—,"квартал, образованный пер. Бойцова, наб. р. Фо...",выявленный объект культурного наследия,Распоряжение КГИОП от 06.08.2019 № 450-р,Памятник,POINT (3374012.280 8382428.690),30.309268,59.922163
1020,9624,—,Депо комплекса построек Товарной станции Варша...,—,"Обводного кан. наб., 118, корп. 12, лит. Ч",выявленный объект культурного наследия,Распоряжение КГИОП от 22.08.2019 № 507-р,Памятник,POINT (3373240.947 8378866.136),30.302339,59.906120


In [63]:
# TODO сохранить слой в географической проекции в формате GeoJSON
gdf_kgiop.to_crs(4326).to_file('kgiop.geojson', driver='GeoJSON')